In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 10 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    EDP : Partie II : Limite des schémas d'ordre 1 et 2
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "seaborn"

# Limite des schémas d'ordre 1 et 2

In [ ]:
def upwind(u, nt, a, dtoverdx):
    for it in range(nt-1):
        flux = a*u
        u[1:] += -dtoverdx * (flux[1:] - flux[:-1])
        u[0] = u[-1]
    return u

def lax_wendroff(u, nt, a, dtoverdx):
    nu = a*dtoverdx
    for it in range(nt-1):        
        flux = u[:-1] + ((1-nu)/2)*(u[1:] - u[:-1])
        u[1:-1] += -dtoverdx * (flux[1:] - flux[:-1])
        u[0] = u[-2]
        u[-1] = u[0]
    return u

## Equation d’advection linéaire

On considère l'équation d'advection linéaire :

$$ \partial_t u(x,t) + a \, \partial_x u(x,t) = 0$$

Nous prendrons comme vitesse de propagation $a=1$.

### Schéma d'ordre 1 pour des données initiales régulières

On choisit une donnée initiale régulière :

$$ u(x,0) = u_0(x) = sin(2 \pi x) $$

In [ ]:
xmin =  0.0
xmax =  1.0

tini = 0.0
tend = 3.1

a = 1.

nx = 400
dx = (xmax-xmin) / nx
x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx)

cfl = 0.5

nt = int(a*(tend-tini)/(cfl*dx)) + 1
dt = (tend-tini)/(nt-1)
dtoverdx = dt/dx

uini = lambda x : np.sin(2*np.pi*x)

# initialisation de la solution avec une maille fantôme à gauche et des conditions aux limtes périodiques
unum = np.zeros(nx+1)
unum[1:] = uini(x)
unum[0] = unum[-1]

unum = upwind(unum, nt, a, dtoverdx)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=uini(x), name='sol initiale'))
fig.add_trace(go.Scatter(x=x, y=uini(x-a*tend), name=f'sol. exa. à t={tend}'))
fig.add_trace(go.Scatter(x=x, y=unum[1:], name=f'sol. num. à t={tend}'))
legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)')
fig.update_layout(height=500, legend=legend)
fig.show()

### Schéma d'ordre 2 pour des données initiales régulières

In [ ]:
xmin =  0.0
xmax =  1.0

tini = 0.0
tend = 3.1

a = 1.0

nx = 400
dx = (xmax-xmin) / nx
x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx)

cfl = 0.5

nt = int(a*(tend-tini)/(cfl*dx)) + 1
dt = (tend-tini)/(nt-1)
dtoverdx = dt/dx

uini = lambda x : np.sin(2*np.pi*x)

# initialisation de la solution avec une maille fantôme à gauche et à droite des conditions aux limtes périodiques
unum = np.zeros(nx+2)
unum[1:-1] = uini(x)
unum[0] = unum[-2]
unum[-1] = unum[0]

unum = lax_wendroff(unum, nt, a, dtoverdx)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=uini(x), name='sol initiale'))
fig.add_trace(go.Scatter(x=x, y=uini(a*x-tend), name=f'sol. exa. à t={tend}'))
fig.add_trace(go.Scatter(x=x, y=unum[:-2], name=f'sol. num. à t={tend}'))
legend = dict(x=0.8, bgcolor='rgba(0,0,0,0)')
fig.update_layout(height=500, legend=legend)
fig.show()

### Schéma d'ordre 1 pour des données initiales discontinues

On choisit une donnée initiale discontinue de type créneau.

In [ ]:
# Solution initiale du créneau
def step_ini(x):
    u = np.empty(x.size)
    for ix, xi in enumerate(x):
        if xi <=-0.5 or xi>=0.5:
            u[ix] = 0.
        else:
            u[ix] = 1.
    return u

In [ ]:
xmin =  -2.0
xmax =  2.0

tini = 0.0
tend = 1.0

a = 1.0

nx = 400
dx = (xmax-xmin) / nx
x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx)

cfl = 0.5

nt = int(a*(tend-tini)/(cfl*dx)) + 1
dt = (tend-tini)/(nt-1)
dtoverdx = dt/dx

# initialisation de la solution avec une maille fantôme à gauche et des conditions aux limtes périodiques
unum = np.zeros(nx+1)
unum[1:] = step_ini(x)
unum[0] = unum[-1]

unum = upwind(unum, nt, a, dtoverdx)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=step_ini(x), name='sol initiale'))
fig.add_trace(go.Scatter(x=x, y=step_ini(x-a*tend), name='sol. exacte'))
fig.add_trace(go.Scatter(x=x, y=unum[1:], name='sol. num.'))
legend = dict(x=0.05, bgcolor='rgba(0,0,0,0)')
fig.update_layout(height=500, legend=legend)
fig.show()

### Schéma d'ordre 2 pour des données initiales discontinues

In [ ]:
xmin =  -2.0
xmax =  2.0

tini = 0.0
tend = 1.0

a = 1.0

nx = 400
dx = (xmax-xmin) / nx
x = np.linspace(xmin+(dx/2), xmax-(dx/2), nx)

cfl = 0.5

nt = int(a*(tend-tini)/(cfl*dx)) + 1
dt = (tend-tini)/(nt-1)
dtoverdx = dt/dx

# initialisation de la solution avec une maille fantôme à gauche et des conditions aux limtes périodiques
unum = np.zeros(nx+2)
unum[1:-1] = step_ini(x)
unum[0] = unum[-2]
unum[-1] = unum[0]

unum = lax_wendroff(unum, nt, a, dtoverdx)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=step_ini(x), name='sol initiale'))
fig.add_trace(go.Scatter(x=x, y=step_ini(x-a*tend), name='sol. exacte'))
fig.add_trace(go.Scatter(x=x, y=unum[1:-1], name='sol. num.'))
legend = dict(x=0.05, bgcolor='rgba(0,0,0,0)')
fig.update_layout(height=500, legend=legend)
fig.show()

Le schéma d'ordre un est très "diffusif" et demande un très grand nombre de points pour converger vers des solutions régulières ou singulières. La convergence pour les méthodes d'ordre un dans le cas singulier est encore plus lente que dans le cas régulier. Passer à l'ordre deux avec un schéma comme Lax-Wendroff est très pertinent pour les solutions régulières mais pose de sérieux problèmes quand la solution est discontinue. Il faut alors analyser la notion de solution singulière : on parle de solution au sens faible ou au sens des distributions et construire des schémas avec de bonnes propriétés de stabilité.